In [1]:
from unified_planning.shortcuts import *
from unified_planning.plans import *
from importlib import reload
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import unifiedplanning_blocksworld as bw
import prompts
import planbench as pb
import policy_model as policy
import llm_utils
import data_loader
import pandas as pd
import regex as re

In [4]:
model='google/gemma-2-9b-it'
temp=0

In [39]:
reload(policy)
reload(prompts)
reload(bw)
reload(pb)
reload(llm_utils)
reload(data_loader)

<module 'data_loader' from '/srv/chawak/planning-with-llms/src/data_loader.py'>

#### Vanilla prompting ####

In [37]:
llm_utils.query_llm('help')

"I'd be happy to help you. Please provide more details about what you need help with, such as:\n\n1. A specific problem you're facing\n2. A question you have\n3. A topic you'd like to discuss\n4. A task you need assistance with\n\nYou can also type one of the following commands to get started:\n\n* `general` for general conversation\n* `math` for math-related help\n* `programming` for programming-related help\n* `writing` for writing-related help\n* `other` for anything else\n\nRemember, I'm here to help and provide guidance to the best of my abilities!"

In [4]:
problem = bw.BlocksworldProblem()

In [5]:
pb.parse_planbench_initial_condition(problem, prompts.nextaction_problem_init)
pb.parse_planbench_goal_state(problem, prompts.nextaction_problem_goal)
#print(problem.initial_values)
#initial vals ko vaapais readable mein translate karna hoga

{clear(blue): true, clear(yellow): true, hand_empty: true, on(blue, orange): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(orange): false, clear(red): false, holding(blue): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [ ]:

pi = policy.PolicyModel()
    # def next_action_one_shot(self, 
    #                          problem_init, problem_goal,problem_action_hist,  # the state S
    #                          example_init, example_goal,example_action_hist,example_next_action)
plan=[]
#action_tuple = pi.next_action_one_shot(prompts.nextaction_problem_init, prompts.nextaction_problem_goal,
#                                       plan,
#                                       prompts.nextaction_example_init, prompts.nextaction_example_goal,
#                                       prompts.nextaction_example_action_hist,prompts.nextaction_example_nextaction)


In [82]:
action_tuple=('unstack','blue','orange')
#print(action_tuple)
next_action=problem.create_action(action=action_tuple[0],blocks=action_tuple[1:])
#print(next_action)

('unstack', 'blue', 'orange')
('blue', 'orange')
unstack(blue, orange)


In [83]:
simulation=problem.create_seq_simulation()
current_state=simulation.get_initial_state()
new_state=problem.validate_action(current_state,next_action,simulation)
#if(new_state):
    #print(new_state)
#else:
    #print('Invalid action proposed by policy')

{on(blue, orange): false, holding(blue): true, clear(orange): true, hand_empty: false, clear(blue): true, clear(yellow): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(red): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [6]:
reload(data_loader)
df=data_loader.load_data("../data/train.parquet")
df.head()


,task,prompt_type,domain,instance_id,example_instance_ids,query,ground_truth_plan
885,task_1_plan_generation,oneshot,blocksworld_3,2,[1],I am playing with a set of blocks where I need...,(pick-up red)\n(stack red orange)\n(pick-up bl...
886,task_1_plan_generation,oneshot,blocksworld_3,3,[2],I am playing with a set of blocks where I need...,(unstack blue orange)\n(put-down blue)\n(unsta...
887,task_1_plan_generation,oneshot,blocksworld_3,4,[3],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
888,task_1_plan_generation,oneshot,blocksworld_3,5,[4],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
889,task_1_plan_generation,oneshot,blocksworld_3,6,[5],I am playing with a set of blocks where I need...,(unstack red orange)\n(put-down red)\n(unstack...


In [156]:
#df.size

4200

In [10]:
def evaluate(attempts,struct,diff_len,results,actions_to_goal,gplan_lens,valid_action_count):
    metrics=pd.DataFrame({
        'Parsable':struct,
        'Attempts':attempts,
        'Terminated':results,
        'Valid Actions':valid_action_count,
        'Distance from goal':actions_to_goal,
        'Diff in plan len':diff_len,
        'GOLD Plan len':gplan_lens
    })
    correct_plans=sum(results)
    c_rate=correct_plans/len(results)
    c_rate=format(c_rate,".2f")
    return metrics,c_rate

In [62]:
#vanilla method
from collections import defaultdict
import math
def main_fullSol(df,model,temp):
    results=[False]*len(df)
    well_struct=[False]*len(df)
    num_tries=[0]*len(df)
    diff_planlen=[-math.inf]*len(df)
    actions_to_goal=[-math.inf]*len(df)
    gplan_lens=[-math.inf]*len(df)
    valid_action_count=[0]*len(df)
    for i in range(0,len(df)):
    #for i in range(0,1):
        
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        
        #declarations: Blocksworld Problem object
        problem = bw.BlocksworldProblem()
        model_plan=None

        #create prompt using PLANBENCH dataset
        #prompt=prompts.make_prompt(df.iloc[i]['query'])
        #print(f'\n\nMake prompt:{prompt} \nEND Make prompt')

        #create prompt using OUR dataset
        prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])

        #parse PLANBENCH dataset into Blocksworld object
        #pbstring=prompts.get_pbstring(df.iloc[i]['query'])
        #init,goal=pb.parse_problem(pbstring)
        #pb.parse_planbench_initial_condition(problem, init)
        #pb.parse_planbench_goal_state(problem, goal)
        #print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        #print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
        
        #changed generation of Blocksworld object for OUR dataset
        pb.parse_planbench_initial_condition(problem, init)
        pb.parse_planbench_goal_state(problem, goal)
        print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')

        #activate solve mode
        pi = policy.PolicyModel()
        #query the llm, parse the actions, return tuples
        tags="Answer within the [PLAN] [PLAN END] tags."
        cot="Step by step reason about the actions you choose to take."
        response=pi.Vanilla_fullSol_one_shot(prompt+"\n"+cot+"\n"+tags,model=model,temp=temp)
        actions=response[0]
        tries=response[1]

        num_tries[i]=tries

        if not actions:
            well_struct[i]=False
            print(f'\n\n LLM exceeded 3 tries')
            results[i]=False
            continue
        well_struct[i]=True
        plan_len=len(actions)
        
        #generate plan from proposed actions
        #actions=[('unstack', 'brown', 'teal'), ('put-down', 'brown'), ('unstack', 'teal', 'violet'), ('put-down', 'teal'), ('unstack', 'violet', 'teal'), ('put-down', 'violet')] 
        #actions=[('unstack', 'green', 'pink'), ('put-down', 'green'), ('unstack', 'pink', 'red'), ('put-down', 'pink'),('pick_up', 'green'), ('stack', 'green' , 'red')] 
        #plan_len=len(actions)

        print(f'\n\nProposed actions by LLM : {actions} \n  END')
        model_plan=problem.create_plan_from_tuples(actions)
        if not model_plan:
            results[i]=False
            print('!!!!!!!!!!!Action statement NOT FROM LIST OF ALLOWED actions!!!!!!!!!!!!')
            #continue

        if model_plan:
            print(f'LLM PLAN: {model_plan}')

        #validate and apply    
        simulation=problem.create_seq_simulation()
        va_counter=0
        valid_state,flag,va_counter=problem.check_and_apply(simulation,model_plan)
        valid_action_count[i]=va_counter
        if not flag:
            results[i]=False
            #continue
            

        #compute actions to goal from last valid state
        # print(f'UBS OBJECT FOR LAST VALID STATE: {valid_state,sim}')
        a2g=problem.actions_to_goal(valid_state)
        actions_to_goal[i]=a2g

        
        #get gold-plan on problem
        gold_plan=problem.generate_plan()
        gplan_actions=str(gold_plan.plan).strip().split('\n')
        gplan_actions=gplan_actions[1:]
        gplan_len=len(gplan_actions)
        gplan_lens[i]=gplan_len
        print(f'\n\n$$$Proposed actions by GOLD PLAN $$$ : {gplan_actions} \n END')


        #termination-check 
        if problem.terminate(valid_state):
            # print(f'Check and Apply return STATE:{valid_state}')
            # print(f'v/s')
            # print(f'GOAL STATE: {problem.goals}')
            # print('Setting result as true')
            results[i]=True
            diff_planlen[i]=plan_len-gplan_len
            continue
        
        results[i]=False

        #compute metrics

    metrics,c_rate=evaluate(num_tries,well_struct,diff_planlen,results,actions_to_goal,gplan_lens,valid_action_count)
    return metrics,c_rate

In [46]:
#load data for PLANBENCH dataset
#df=data_loader.load_data("../data/train.parquet")
#df.head()
#results=main_fullSol(df)

In [47]:
#print(f"\n\n Model solution's verdict: {results}")

In [48]:
#df.head

In [37]:
#load data for OUR dataset
import pandas as pd
n=3
df=pd.read_csv(f'../data/{n}_blocks/Oneshot_val_{n}_blocks')
df.drop(columns='Unnamed: 0',inplace=True)
df

,init,goal,demo_init,demo_goal,demo_plan
0,"(('violet', 'teal', 'brown'),)","(('violet',), ('teal',), ('brown',))","the green block is clear, the hand is empty, ...","the green block is on the table, the red bloc...",[PLAN]\nunstack the green block from on top of...
1,"(('red', 'pink', 'green'),)","(('pink',), ('red', 'green'))","the pink block is clear, the red block is clea...","the red block is on the table, the pink block...",[PLAN]\nunstack the pink block from on top of ...
2,"(('pink', 'red', 'green'),)","(('red', 'pink'), ('green',))","the violet block is clear, the hand is empty,...","the teal block is on the table, the violet bl...",[PLAN]\nunstack the violet block from on top o...
3,"(('teal', 'violet', 'brown'),)","(('violet',), ('teal',), ('brown',))","the brown block is clear, the hand is empty, ...","the violet block is on the table, the teal bl...",[PLAN]\nunstack the brown block from on top of...
4,"(('pink',), ('green', 'red'))","(('pink',), ('red', 'green'))","the brown block is clear, the hand is empty, ...","the brown block is on the table, the violet b...",[PLAN]\nunstack the brown block from on top of...
5,"(('pink', 'green'), ('red',))","(('red', 'pink'), ('green',))","the violet block is clear, the brown block is ...","the violet block is on the table, the brown b...",[PLAN]\nunstack the violet block from on top o...
6,"(('violet',), ('teal',), ('brown',))","(('brown', 'teal', 'violet'),)","the pink block is clear, the red block is clea...","the pink block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
7,"(('brown', 'violet'), ('teal',))","(('teal', 'brown', 'violet'),)","the pink block is clear, the green block is cl...","the green block is on the table, the red bloc...",[PLAN]\nunstack the green block from on top of...
8,"(('pink',), ('red', 'green'))","(('green', 'pink'), ('red',))","the red block is clear, the green block is cle...","the green block is on the table, the pink blo...",[PLAN]\nunstack the red block from on top of t...
9,"(('green', 'pink'), ('red',))","(('red', 'pink'), ('green',))","the violet block is clear, the teal block is c...","the violet block is on the table, the brown b...",[PLAN]\nunstack the teal block from on top of ...


In [50]:
# df=df[1:2]
# df

In [51]:
df.shape

(15, 5)

In [52]:
model='google/gemma-2-9b-it'
temp=0

In [69]:
response=main_fullSol(df,model=model,temp=temp)



--------------------Entering main loop PROBLEM #0--------------------
State in create_ic (('violet', 'teal', 'brown'),)
State in create_goal (('violet',), ('teal',), ('brown',))


Blocksworld Problem Initial Values:{clear(brown): true, hand_empty: true, on_table(violet): true, on(teal, violet): true, on(brown, teal): true, on(brown, brown): false, on(violet, brown): false, on(teal, brown): false, on(brown, violet): false, on(violet, violet): false, on(violet, teal): false, on(teal, teal): false, clear(violet): false, clear(teal): false, on_table(brown): false, on_table(teal): false, holding(brown): false, holding(violet): false, holding(teal): false}

Blocksworld Problem Goal Condition:[on_table(violet), on_table(teal), on_table(brown)]
*************** PROMPT ***************: 

 I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a

In [70]:
metrics_df=response[0]
c_rate=response[1]

In [55]:
#metrics_df=pd.read_csv('../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [71]:
print(f'----------------{model}, w COT, w demo help, w plan-end tags----------')
metrics_df

----------------google/gemma-2-9b-it, w COT, w demo help, w plan-end tags----------


,Parsable,Attempts,Terminated,Valid Actions,Distance from goal,Diff in plan len,GOLD Plan len
0,True,1,True,6,0,2.0,4
1,True,1,False,1,5,-inf,6
2,True,1,True,6,0,0.0,6
3,True,1,False,0,4,-inf,4
4,True,1,True,6,0,2.0,4
5,True,1,True,6,0,2.0,4
6,True,1,False,3,7,-inf,4
7,True,1,True,6,0,0.0,6
8,True,1,True,6,0,2.0,4
9,True,3,False,1,1,-inf,2


In [72]:
print(f'Correctness rate : {c_rate}')
print(f"Goal State Reached: {sum(metrics_df['Distance from goal']==0)}")

Correctness rate : 0.53
Goal State Reached: 8


In [330]:
#metrics_df.to_csv(f'../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [20]:
problem=bw.BlocksworldProblem()

In [38]:
BLOCK='green'
blocks={'green':'green'}
block=blocks.get(BLOCK)
block

'green'

In [90]:
len(df)

15

In [ ]:
#qwen
Qwen/Qwen2-7B-Instruct
Qwen/Qwen2.5-7B-Instruct

#llama
meta-llama/Meta-Llama-3-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo

#mistral
mistralai/Mistral-7B-Instruct-v0.1
mistralai/Mistral-7B-Instruct-v0.2
mistralai/Mistral-7B-Instruct-v0.3

#phi-4
microsoft/Phi-4-multimodal-instruct

#gemma
google/gemma-1.1-7b-it
google/gemma-3-4b-it
google/gemma-2-9b-it

#### COT: Trials ####

In [18]:
reasons=[]

In [5]:
reason='''I faced the stubborn tower: Brown on Violet, Violet on Teal, Teal on table. The goal? Brown on table, Violet on Brown, Teal on Violet. My fingers flexed.

"Brown's clear—I'll move it first." I unstuck Brown from Violet, cradling it. Wait. The goal wants Brown on table... so far so good. I set Brown down with a clunk.

Now Violet sat exposed atop Teal. "Need Violet on Brown... but first, is Violet clear?" I checked—yes, no blocks above. I unstacked Violet, then hesitated. "Brown's ready—but is it still clear?" I nudged it. Good, untouched. I stacked Violet onto Brown.

Teal remained alone on the table. "Now for the finale..." I reached, then froze. "Is Violet clear for Teal?" I peered—Violet's surface bare. Safe. I lifted Teal, stacked it atop Violet.

The new tower stood: Brown (table) → Violet → Teal.

Not Just Moving Blocks:
Each action trailed by silent questions—"Can I? Should I?"—until the blocks aligned not by speed, but by relentless verification.

Moral: Precision isn't in the moves; it's in the moments between.

'''

In [35]:
reasons.append(reason)

In [36]:
i=1
for r in reasons:
    print(f'{i}. {r}')
    i+=1

1. I see the blocks: Red at the bottom, Pink in the middle, Green on top. My goal? Rearrange them to Green, Red, Pink.

"Green is free to move first," I note, my hand hovering over the stack. Carefully, I lift Green off Pink and place it gently on the table.

"Now Pink is clear," I observe. With steady movements, I pick up Pink and set it down next to Green. Red sits alone now, waiting for its turn.

"Time to build the new stack," I think. I pick up Red - "Is Green clear and ready?" Yes. I place Red securely on top of Green.

One last check: "Is Red clear for Pink?" Confirming it is, I carefully lift Pink and stack it on Red. The new tower stands perfect - Green supporting Red, with Pink proudly on top.

"Mission complete," I confirm to myself. "Every move planned, every placement verified."

Lesson: When I take time to think through each step, success follows naturally.



2. I examine my blocks: Pink sits perfectly on Green, both untouched. Red waits patiently on the table, clear and

In [6]:
prompt=('''I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block on top of another block if I am holding the block being stacked
        I can only stack a block on top of another block if the block onto which I am stacking the block is clear
        Once I put down or stack a block, my hand becomes empty
        Once you stack a block on top of a second block, the second block is no longer clear

[STATEMENT]
As initial conditions I have that, the green block is clear,  the hand is empty, the red block is on the table, the pink block is on top of the red block, the green block is on top of the pink block.
My goal is to have that  the green block is on the table, the red block is on top of the green block, the pink block is on top of the red block.

Wait, let's think step by step.
[REASONING]
'''

f"\n \n{reason}\n\n [REASONING END]"
'''
My plan is as follows:

[PLAN]
unstack the green block from on top of the pink block
put down the green block
unstack the pink block from on top of the red block
put down the pink block
pick up the red block
stack the red block on top of the green block
pick up the pink block
stack the pink block on top of the red block
[PLAN END]

[STATEMENT]
As initial conditions I have that, the brown block is clear,  the hand is empty, the violet block is on the table, the teal block is on top of the violet block, the brown block is on top of the teal block.
My goal is to have that  the violet block is on the table, the teal block is on the table, the brown block is on the table.

Wait, let's think step by step.
[REASONING]

Reason about the steps you take. Finally, answer within the [PLAN] [PLAN END] tags.''')


In [42]:
results=[]
for i in range(0,5):
    prompt,init,goal=prompts.cot_make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'],reasons[i])
    print(f'----------------------------------PROMPT {i}------------------------:\n\n{prompt}')
    r=llm_utils.query_llm(prompt,model=model,temperature=temp)
    results.append(r)

State in create_ic (('violet', 'teal', 'brown'),)
State in create_goal (('violet',), ('teal',), ('brown',))
----------------------------------PROMPT 0------------------------:

I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I a

In [46]:
for r in results:
    print(r)

[REASONING]
Okay, let's analyze the situation. We have Brown on top of Teal, which is on top of Violet. Violet is on the table. The goal is to have all three blocks on the table. This means we need to unstack them and put them down individually.

First, I'll unstack the brown block from the teal block and put it on the table. Then, I'll unstack the teal block from the violet block and put it on the table. Finally, the violet block is already on the table, so no action is needed for it.

[PLAN]
unstack the brown block from on top of the teal block
put down the brown block
unstack the teal block from on top of the violet block
put down the teal block
[PLAN END]
[REASONING]
Okay, let's analyze the situation and devise a plan to achieve the goal. The current state is: green is clear, hand is empty, red is on the table, pink is on red, and green is on pink. The goal is: pink is on the table, red is on the table, and green is on red.

The key here is to move the blocks around to achieve the 

In [5]:
import pandas as pd
from io import StringIO

In [18]:
# Clean up leading/trailing spaces
markdown_table = markdown_table.strip()

# Read it as if it's a CSV, but with "|" separator
df = pd.read_csv(StringIO(markdown_table), sep="|", engine="python")

# Drop the weird first and last empty columns (caused by the leading/trailing '|')
df = df.drop(columns=["Unnamed: 0", "Unnamed: 9"])

# Optionally strip spaces from column names
df.columns = df.columns.str.strip()

print(df)

    Index      Parsable  Attempts      Terminated  Valid Actions  \
0       0   True                1   False                      4   
1       1   True                1   True                       2   
2       2   True                1   True                       4   
3       3   True                1   False                      0   
4       4   True                1   True                       6   
..    ...           ...       ...             ...            ...   
94     94   True                1   False                      0   
95     95   True                1   False                      0   
96     96   True                1   False                      0   
97     97   True                1   True                       4   
98     98   True                1   True                       4   

    Distance from goal      Diff in plan len  GOLD Plan len  
0                    2                 -inf               2  
1                    0                    0               2

In [20]:
df.drop(columns=['Index'],inplace=True)

In [23]:
df.head(11)

,Parsable,Attempts,Terminated,Valid Actions,Distance from goal,Diff in plan len,GOLD Plan len
0,True,1,False,4,2,-inf,2
1,True,1,True,2,0,0,2
2,True,1,True,4,0,0,4
3,True,1,False,0,4,-inf,4
4,True,1,True,6,0,0,6
5,True,1,False,2,6,-inf,8
6,True,1,True,4,0,0,4
7,True,1,False,0,4,-inf,4
8,True,1,False,2,4,-inf,6
9,True,1,False,4,2,-inf,6


In [ ]:
df.to_csv('')

In [7]:
df=df.rename(columns=
          {':-----------':'Parsable',
           '-----------:':'Attempts',
           ':-------------':'Terminated',
           '----------------:':'Valid Actions',
           '---------------------:':'Distance from goal',
           '-------------------:':'Diff in plan len',
           '----------------:.1':'GOLD Plan len',})

NameError: name 'df' is not defined

In [12]:
df

,Parsable,Attempts,Terminated,Valid Actions,Distance from goal,Diff in plan len,GOLD Plan len
0,True,1,True,6,0,0,6
1,True,1,False,1,13,-inf,8
2,True,1,False,0,8,-inf,8
3,True,1,True,10,0,4,6
4,True,1,False,2,6,-inf,8
...,...,...,...,...,...,...,...
961,True,1,False,0,10,-inf,10
962,True,2,True,8,0,0,8
963,True,1,False,1,11,-inf,12
964,True,1,False,5,3,-inf,8


In [3]:
split='train'
n=3

In [74]:
#df.to_csv(f'../results/COT_Prompting_w_good/{split}_{n}_blocks')

In [4]:
#df=pd.read_csv(f'../results/COT_Prompting_w_good/{split}_{n}_blocks').drop(columns='Unnamed: 0')
#df=pd.read_csv(f'../results/COT_prompting_w_not_so_good/{split}_{n}_blocks').drop(columns='Unnamed: 0')
#df=pd.read_csv(f'../results/Vanilla_Prompting/Oneshot_{split}_{n}_blocks').drop(columns='Unnamed: 0')
df=pd.read_csv(f'../results/DeepThink_COT_{split}_{n}_blocks').drop(columns='Unnamed: 0')
df

,Parsable,Attempts,Terminated,Valid Actions,Distance from goal,Diff in plan len,GOLD Plan len
0,True,1,False,4,2,-inf,2
1,True,1,True,2,0,0.0,2
2,True,1,True,4,0,0.0,4
3,True,1,False,0,4,-inf,4
4,True,1,True,6,0,0.0,6
...,...,...,...,...,...,...,...
94,True,1,False,0,6,-inf,6
95,True,1,False,0,6,-inf,6
96,True,1,False,0,2,-inf,2
97,True,1,True,4,0,0.0,4


In [5]:
c=sum(df['Terminated'])

In [6]:
c

37

In [7]:
c/len(df['Terminated'])

0.37373737373737376

In [58]:
#df['Parsable']=df['Parsable'].str.strip().map({'True':True, 'False':False})

#### Local LLM set-up ####

Strategise at each step before you choose an action, then form a final plan.
Step by step reason about the actions you choose to take.

In [10]:
prompt1='''I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block on top of another block if I am holding the block being stacked
        I can only stack a block on top of another block if the block onto which I am stacking the block is clear
        Once I put down or stack a block, my hand becomes empty
        Once you stack a block on top of a second block, the second block is no longer clear

[STATEMENT]
As initial conditions I have that, the green block is clear,  the hand is empty, the pink block is on the table, the red block is on top of the pink block, the green block is on top of the red block.
My goal is to have that  the green block is on the table, the pink block is on top of the green block, the red block is on the table.

My plan is as follows:

[PLAN]
unstack the green block from on top of the red block
put down the green block
unstack the red block from on top of the pink block
put down the red block
pick up the pink block
stack the pink block on top of the green block
[PLAN END]

[STATEMENT]
As initial conditions I have that, the red block is clear, the green block is clear,  the hand is empty, the pink block is on the table, the green block is on top of the pink block, the red block is on the table.
My goal is to have that  the pink block is on the table, the red block is on top of the pink block, the green block is on the table.

[PLAN]
Answer within the [PLAN] [PLAN END] tags.
'''

In [7]:
import torch
#torch.cuda.empty_cache()
model_name='google/gemma-2-9b-it'
temp=0

In [10]:
#model,tokenizer=llm_utils.get_model_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
response=llm_utils.query_local_model(model=model,tokenizer=tokenizer,model_name=model_name,temperature=0,prompt=prompt1)

PROMPT: user: I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only st

In [14]:
r=llm_utils.query_llm(model=model_name,temperature=temp,prompt=prompt)

Querying model : google/gemma-2-9b-it with temperature: 0


In [15]:
print(r)

Okay, let's analyze the situation and devise a plan to achieve the goal.

The initial state is: brown clear, hand empty, violet on table, teal on violet, brown on teal.
The goal state is: violet on table, teal on table, brown on table.

This means we need to separate the blocks and place them individually on the table.

[PLAN]
unstack the brown block from on top of the teal block
put down the brown block
unstack the teal block from on top of the violet block
put down the teal block
[PLAN END]


In [17]:
r=llm_utils.query_llm(prompt=prompt1+"Let's DeepThink this.",model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B',temperature=0,max_tokens=1200)

Querying model : deepseek-ai/DeepSeek-R1-Distill-Llama-70B with temperature: 0


In [18]:
print(r)

<think>
Okay, so I'm trying to figure out how to rearrange these blocks according to the given goal. Let me start by understanding the initial setup and the desired outcome.

In the initial state, the red block is clear, the green block is clear, the hand is empty. The pink block is on the table, the green block is on top of the pink block, and the red block is on the table. So, the current stack is pink with green on top, and red is separate on the table.

The goal is to have the pink block on the table, the red block on top of the pink block, and the green block on the table. So, I need to move the red block onto the pink block and have the green block by itself on the table.

Let me think about the possible actions I can take. The available actions are pick up, unstack, put down, and stack. But there are several restrictions. I can only pick up or unstack one block at a time, and only if my hand is empty. Also, I can only pick up a block if it's on the table and clear. A block is cl

In [22]:
demo_prompt=prompt1.split('[PLAN]')[0]
print(demo_prompt)

I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block on

In [5]:
train_3=pd.read_csv(f'../data/3_blocks/SFT_train_3_blocks_fullPlan').drop(columns='Unnamed: 0')

In [6]:
def get_deep_thoughts(df,model,temp):
    thoughts=[]
    for i in range(len(df)):
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        #prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])
        prompt=df['prompt'][i]
        demo_prompt=prompt.split('[PLAN]')[0].strip()
        demo_prompt=demo_prompt+'[PLAN]'
        r=llm_utils.query_llm(prompt=demo_prompt+"Let's DeepThink this.",model=model,temperature=temp,max_tokens=1200)
        thought=r.split('<think>')[1].split('</think>')[0].strip()
        thoughts.append(thought)
    return thoughts


In [7]:
model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B'
temp=0

In [8]:
thoughts=get_deep_thoughts(train_3,model,temp)



--------------------Entering main loop PROBLEM #0--------------------
Querying model : deepseek-ai/DeepSeek-R1-Distill-Llama-70B with temperature: 0


KeyboardInterrupt: 

In [33]:
thoughts

["Okay, so I'm trying to figure out how to rearrange these blocks from the initial setup to the goal setup. Let me start by understanding the problem step by step.\n\nFirst, the initial state is:\n- Green block is clear.\n- Hand is empty.\n- Red block is on the table.\n- Pink block is on top of the red block.\n- Green block is on top of the pink block.\n\nSo, the stack from bottom to top is: Red → Pink → Green.\n\nThe goal is to have:\n- Green block on the table.\n- Red block on top of the green.\n- Pink block on top of the red.\n\nSo, the desired stack from bottom to top is: Green → Red → Pink.\n\nAlright, so I need to move the blocks from the initial stack to this new configuration. Let me think about the possible actions I can take, considering the restrictions.\n\nThe actions allowed are:\n1. Pick up a block.\n2. Unstack a block from on top of another.\n3. Put down a block.\n4. Stack a block on top of another.\n\nRestrictions:\n- Can only pick up or unstack one block at a time.\n- 

In [36]:
thoughts_df=pd.DataFrame(thoughts,columns=['thought'])

In [38]:
thoughts_df.to_csv('../data/3_blocks/thoughts.csv')

In [ ]:
Parsable',
      3            '-----------:':'Attempts',
      4            ':-------------':'Terminated',
      5            '----------------:':'Valid Actions',
      6            '---------------------:':'Distance from goal',
      7            '-------------------:':'Diff in plan len',
      8            '----------------:.1':'GOLD Plan len',})

In [8]:
markdown_table = '''
|Index|Parsable|Attempts|Terminated|Valid Actions|Distance from goal|Diff in plan len|GOLD Plan len|
|  0 | True       |          1 | False        |               4 |                    2 |               -inf |               2 |
|  1 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
|  2 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
|  3 | True       |          1 | False        |               0 |                    4 |               -inf |               4 |
|  4 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
|  5 | True       |          1 | False        |               2 |                    6 |               -inf |               8 |
|  6 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
|  7 | True       |          1 | False        |               0 |                    4 |               -inf |               4 |
|  8 | True       |          1 | False        |               2 |                    4 |               -inf |               6 |
|  9 | True       |          1 | False        |               4 |                    2 |               -inf |               6 |
| 10 | True       |          1 | False        |               8 |                    2 |               -inf |               6 |
| 11 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 12 | True       |          1 | False        |               4 |                    6 |               -inf |               4 |
| 13 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 14 | True       |          1 | False        |               4 |                    2 |               -inf |               4 |
| 15 | True       |          1 | False        |               4 |                    6 |               -inf |               6 |
| 16 | True       |          1 | False        |               4 |                    6 |               -inf |               2 |
| 17 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 18 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 19 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 20 | True       |          1 | False        |               4 |                    2 |               -inf |               6 |
| 21 | True       |          1 | False        |               5 |                    3 |               -inf |               2 |
| 22 | True       |          1 | False        |               2 |                    4 |               -inf |               6 |
| 23 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 24 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 25 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 26 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 27 | True       |          1 | False        |               0 |                    4 |               -inf |               4 |
| 28 | True       |          1 | True         |               6 |                    0 |                  4 |               2 |
| 29 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 30 | True       |          1 | True         |               8 |                    0 |                  0 |               8 |
| 31 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 32 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 33 | True       |          1 | False        |               1 |                    7 |               -inf |               6 |
| 34 | True       |          1 | False        |               3 |                    7 |               -inf |               4 |
| 35 | True       |          1 | False        |               1 |                    7 |               -inf |               8 |
| 36 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 37 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 38 | True       |          1 | False        |               0 |                    4 |               -inf |               4 |
| 39 | True       |          1 | True         |               4 |                    0 |                  1 |               4 |
| 40 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 41 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 42 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 43 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 44 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 45 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 46 | True       |          1 | False        |               2 |                    6 |               -inf |               8 |
| 47 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 48 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 49 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 50 | True       |          1 | False        |               1 |                    3 |               -inf |               4 |
| 51 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 52 | True       |          1 | False        |               4 |                    8 |               -inf |               4 |
| 53 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 54 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 55 | True       |          1 | False        |               3 |                    3 |               -inf |               2 |
| 56 | True       |          1 | False        |               0 |                    4 |               -inf |               4 |
| 57 | True       |          1 | False        |               2 |                    4 |               -inf |               6 |
| 58 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 59 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 60 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 61 | True       |          1 | False        |               4 |                    2 |               -inf |               6 |
| 62 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 63 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 64 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 65 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 66 | True       |          1 | True         |               8 |                    0 |                  0 |               8 |
| 67 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 68 | True       |          1 | False        |               1 |                    3 |               -inf |               4 |
| 69 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 70 | True       |          1 | False        |               4 |                    6 |               -inf |               8 |
| 71 | True       |          1 | False        |               1 |                    3 |               -inf |               2 |
| 72 | True       |          1 | False        |               1 |                    5 |               -inf |               4 |
| 73 | True       |          1 | False        |               6 |                    6 |               -inf |               8 |
| 74 | True       |          1 | False        |               1 |                    7 |               -inf |               6 |
| 75 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 76 | True       |          1 | False        |               3 |                    3 |               -inf |               2 |
| 77 | True       |          1 | False        |               1 |                    7 |               -inf |               8 |
| 78 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 79 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 80 | True       |          1 | False        |               6 |                    8 |               -inf |               6 |
| 81 | True       |          1 | False        |               1 |                    5 |               -inf |               6 |
| 82 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 83 | True       |          1 | True         |               2 |                    0 |                  0 |               2 |
| 84 | True       |          1 | False        |               0 |                    8 |               -inf |               8 |
| 85 | True       |          1 | False        |               1 |                    3 |               -inf |               4 |
| 86 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 87 | True       |          1 | False        |               1 |                    7 |               -inf |               6 |
| 88 | True       |          1 | True         |               8 |                    0 |                  0 |               8 |
| 89 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 90 | True       |          1 | True         |               6 |                    0 |                  0 |               6 |
| 91 | True       |          1 | False        |               0 |                    2 |               -inf |               2 |
| 92 | True       |          1 | False        |               1 |                    1 |               -inf |               2 |
| 93 | True       |          1 | False        |               2 |                    4 |               -inf |               6 |
| 94 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 95 | True       |          1 | False        |               0 |                    6 |               -inf |               6 |
| 96 | True       |          1 | False        |               0 |                    2 |               -inf |               2 |
| 97 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |
| 98 | True       |          1 | True         |               4 |                    0 |                  0 |               4 |'''